<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №1</center>

<center>«Введение в MapReduce модель на Python»</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>



<br>
<p style="text-align:right;">Обучающийся: <Вернер К.А.>
<br>6133-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2024</center>


# Введение в MapReduce модель на Python


In [102]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [103]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [104]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [105]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [106]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [107]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [108]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [109]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [110]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [111]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [112]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [113]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [114]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [115]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [116]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.022447506316719),
 (1, 2.022447506316719),
 (2, 2.022447506316719),
 (3, 2.022447506316719),
 (4, 2.022447506316719)]

## Inverted index

In [117]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [118]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [119]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [120]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCE and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18)]),
 (1, [('is', 18), ('what', 10)])]

## TeraSort

In [121]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.0718855851124236),
   (None, 0.08641212856837699),
   (None, 0.08869742825120153),
   (None, 0.1685828133365158),
   (None, 0.186342718478392),
   (None, 0.21401222397549013),
   (None, 0.2190533401604552),
   (None, 0.22009281314354745),
   (None, 0.22770516027732945),
   (None, 0.24021290445802002),
   (None, 0.3079654463068926),
   (None, 0.33997767925640454),
   (None, 0.3544088129509023),
   (None, 0.37423738247676996),
   (None, 0.38902398773466373)]),
 (1,
  [(None, 0.5825816752502369),
   (None, 0.6731883598036665),
   (None, 0.6938816787760177),
   (None, 0.7357685280986299),
   (None, 0.7766309123628762),
   (None, 0.7955049422319969),
   (None, 0.8260392784966926),
   (None, 0.8374040362871285),
   (None, 0.927325900472885),
   (None, 0.9315496081995123),
   (None, 0.9326438511327864),
   (None, 0.9542990383533005),
   (None, 0.9594945197843646),
   (None, 0.9631259963486904),
   (None, 0.9855097577325576)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [122]:
import numpy as np

# Генерация списка случайных чисел
split = [np.random.randint(30) for _ in range(15)]
print("Сгенерированный список:", split)

# Функция RECORDREADER, которая возвращает список значений
def RECORDREADER():
    return [(0, split[n]) for n in range(len(split))]  # Возвращаем список значений напрямую

# Функция MAP, которая возвращает элементы как кортежи
def MAP(n, spl):
    yield (n, spl)  # Возвращаем кортеж с элементом

# Функция REDUCE, которая находит максимальное значение
def REDUCE(n, spl):
    yield (n, max(spl))  # Возвращаем максимальное значение из списка

# Запуск MapReduce
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print("Максимальное значение:", output)  # Выводим результат

Сгенерированный список: [26, 13, 21, 4, 0, 16, 27, 11, 25, 25, 1, 4, 29, 17, 17]
Максимальное значение: [(0, 29)]


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [123]:
import numpy as np

storage = [np.random.randint(30) for _ in range(15)]
print(storage)

def RECORDREADER():
    return [(0, storage[n]) for n in range(len(storage))]

def MAP(n, stor):
    yield (n, stor)

def REDUCE(n, stor):
    yield (n, sum(stor) / len(stor))


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(output)

[3, 6, 4, 4, 19, 7, 0, 0, 21, 19, 24, 14, 11, 9, 28]
[(0, 11.266666666666667)]


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [124]:
def groupbykey(sortied):
  sortied = [(num, num) for num in sorted(sortied)]
  t = {}
  for k1, v1 in sortied:
    t[k1] = t.get(k1, []) + [v1]
  return t.items()
sortied = [1,5,2,2,4,8,8,9,0]
output = groupbykey(sortied)
print(list(output))

[(0, [0]), (1, [1]), (2, [2, 2]), (4, [4]), (5, [5]), (8, [8, 8]), (9, [9])]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [125]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

maps = 3
reducers = 3

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split): #enumerate встроенная функция которая добавляет к счетчик к итерируемому объекту и возвращает его
            yield (userindex, user)

    split_size = int(np.ceil(len(input_collection)/ maps)) #Функция ceil округляет в большую сторону числа
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])

def MAP(_, user:str):
    yield (user.id, user)

def REDUCE(user_id: int, users: Iterator[int]):
    yield users[0]

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(partitioned_output)

6 key-value pairs were sent over a network.
[(0, [User(id=0, age=55, social_contacts=20, gender='male'), User(id=3, age=33, social_contacts=800, gender='female')]), (1, [User(id=1, age=25, social_contacts=240, gender='female')]), (2, [User(id=2, age=25, social_contacts=500, gender='female')])]


## Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [126]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row: NamedTuple):
    if row.social_contacts > 300:
        yield (row, row)

def REDUCE(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output


[((User(id=2, age=25, social_contacts=500, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')),
  [(User(id=2, age=25, social_contacts=500, gender='female'),
    User(id=2, age=25, social_contacts=500, gender='female'))]),
 ((User(id=3, age=33, social_contacts=800, gender='female'),
   User(id=3, age=33, social_contacts=800, gender='female')),
  [(User(id=3, age=33, social_contacts=800, gender='female'),
    User(id=3, age=33, social_contacts=800, gender='female'))])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [127]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

def MAP(_, rec):
    yield (rec.age, rec.age)

def REDUCE(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[((25, 25), [(25, 25), (25, 25)]),
 ((33, 33), [(33, 33)]),
 ((55, 55), [(55, 55), (55, 55)])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [128]:
input_collection_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=4, age=33, gender="male", social_contacts=973),
]

def MAP(_, value):
    yield (value, None)

def REDUCE(key, value):
    yield (key, None)

def RECORDREADER():
    return [(u.id, u) for u in input_collection_1 + input_collection_2]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[((User(id=0, age=55, social_contacts=20, gender='male'), None), None),
 ((User(id=1, age=25, social_contacts=240, gender='female'), None), None),
 ((User(id=2, age=25, social_contacts=500, gender='female'), None), None),
 ((User(id=3, age=33, social_contacts=800, gender='female'), None), None),
 ((User(id=4, age=33, social_contacts=973, gender='male'), None), None)]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [129]:
def MAP(_, row: NamedTuple):
    yield (row.id, row)

def REDUCE(row_id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows

def RECORDREADER():
    return [(u.id, u) for u in input_collection_1 + input_collection_2]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[[(2, User(id=2, age=25, social_contacts=500, gender='female')),
  (2, User(id=2, age=25, social_contacts=500, gender='female'))],
 [(3, User(id=3, age=33, social_contacts=800, gender='female')),
  (3, User(id=3, age=33, social_contacts=800, gender='female'))]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [130]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

input_collection_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=4, age=33, gender="male", social_contacts=973),
]

def RECORDREADER():
    return [(0, a) for a in input_collection_1] + [(0, b) for b in input_collection_2]

def MAP(collection_id, user):
    yield (user, collection_id)

def REDUCE(user, collections):
    if collections == [0]:
        yield user

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[User(id=0, age=55, social_contacts=20, gender='male'),
 User(id=1, age=25, social_contacts=240, gender='female'),
 User(id=4, age=33, social_contacts=973, gender='male')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [131]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    job_id: int

class Job(NamedTuple):
    id: int
    name: str

users_collection = [
    User(id=1, age=25, gender="male", social_contacts=20, job_id=1),
    User(id=2, age=25, gender="female", social_contacts=240, job_id=2),
    User(id=3, age=22, gender="male", social_contacts=642, job_id=2),
    User(id=4, age=54, gender="female", social_contacts=123, job_id=3),
    User(id=5, age=24, gender="female", social_contacts=800, job_id=1),
]

job_collection = [
    Job(id=1, name="Doctor"),
    Job(id=2, name="Nerce"),
    Job(id=3, name="Teacher"),
    Job(id=4, name="Driver"),
]

def MAP(job_id, row):
    yield (job_id, row)

def REDUCE(job_id, rows):
    users = []
    job = None
    for row in rows:
        if type(row) is User:
            users += [row]
        else:
            job = row
    for row in rows:
        if type(row) is User:
          #(a,b,c), где a - юзер, b - id профессии, c - профессия
            yield (row, row.job_id, job)

def RECORDREADER():
    return [(user.job_id, user) for user in users_collection] + [(job.id, job) for job in job_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=1, age=25, social_contacts=20, gender='male', job_id=1),
  1,
  Job(id=1, name='Doctor')),
 (User(id=5, age=24, social_contacts=800, gender='female', job_id=1),
  1,
  Job(id=1, name='Doctor')),
 (User(id=2, age=25, social_contacts=240, gender='female', job_id=2),
  2,
  Job(id=2, name='Nerce')),
 (User(id=3, age=22, social_contacts=642, gender='male', job_id=2),
  2,
  Job(id=2, name='Nerce')),
 (User(id=4, age=54, social_contacts=123, gender='female', job_id=3),
  3,
  Job(id=3, name='Teacher'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [132]:
def MAP(job_id, user, job):
    yield (job_id, user)

def REDUCE(job_id, rows):
    yield f"job_id={job_id} has {len(rows)} user(s)"

def RECORDREADER():
    return [(job_id, user, job) for user, job_id, job in output]

output1 = MapReduce(RECORDREADER, MAP, REDUCE)
output1 = list(output1)
output1

['job_id=1 has 2 user(s)', 'job_id=2 has 2 user(s)', 'job_id=3 has 1 user(s)']

## Вычисление TF-IDF (Term Frequency – Inverse Document Fraquency)

Реализуется в три этапа:

**Этап 1:** Частота слова в документе

**Этап 2:** Количество документов, в которых встречается слово

**Этап 3:** Расчёт TF-IDF

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример документов
documents = [
    "Кошка сидит на ковре",
    "Собака лежит на полу",
    "Кошка и собака играют вместе",
    "Кошка ловит мышь"
]

# Создание объекта TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразование документов в матрицу TF-IDF
tfidf_matrix = vectorizer.fit_transform(documents)

# Получение словаря терминов
feature_names = vectorizer.get_feature_names_out()

# Преобразование матрицы в массив
dense = tfidf_matrix.todense()

# Преобразование в DataFrame для удобства
import pandas as pd

df_tfidf = pd.DataFrame(dense, columns=feature_names)

# Вывод результата
print(df_tfidf)

    вместе   играют    ковре     кошка     лежит     ловит      мышь  \
0  0.00000  0.00000  0.57458  0.366747  0.000000  0.000000  0.000000   
1  0.00000  0.00000  0.00000  0.000000  0.555283  0.000000  0.000000   
2  0.57458  0.57458  0.00000  0.366747  0.000000  0.000000  0.000000   
3  0.00000  0.00000  0.00000  0.411378  0.000000  0.644503  0.644503   

         на      полу    сидит    собака  
0  0.453005  0.000000  0.57458  0.000000  
1  0.437791  0.555283  0.00000  0.437791  
2  0.000000  0.000000  0.00000  0.453005  
3  0.000000  0.000000  0.00000  0.000000  


### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [134]:
from typing import Iterator
import numpy as np

n = 4
mat = np.ones((5, n))
vec1 = np.random.rand(n // 2)
vec2 = np.random.rand(n - n // 2)

def MAP(coordinates:(int, int), value:int):
    i, j = coordinates
    if (j < len(vec1)):
        yield (i, value * vec1[j])
    else:
        yield (i, value * vec2[j - len(vec1)])

def REDUCE(i:int, products:Iterator[NamedTuple]):
    sum = 0
    for p in products:
        sum += p
    yield (i, sum)

def RECORDREADER():
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            yield ((i, j), mat[i, j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.495275180979448),
 (1, 2.495275180979448),
 (2, 2.495275180979448),
 (3, 2.495275180979448),
 (4, 2.495275180979448)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [135]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [136]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  for i in range(small_mat.shape[0]):
    yield ((i, k), small_mat[i][j] * v1)

def REDUCE(key, values):
  yield (key, sum(values))

Проверьте своё решение

In [137]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [138]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [139]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
  for i in range (small_mat.shape[0]):
    for j in range(big_mat.shape[0]):
      for k in range(big_mat.shape[1]):
        yield (((i, j), small_mat[i,j]), ((j, k), big_mat[j,k]))

def MAP(mat1, mat2):
  (k1, v1) = mat1
  (k2, v2) = mat2
  (i, _) = k1
  (_, k) = k2
  yield ((i, k), v1 * v2)

def REDUCE(key, values):
  yield (key, sum(values))

# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat

np.allclose(reference_solution, asmatrix(solution))  # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [140]:
maps = 2
reducers = 2

def INPUTFORMAT():
    global maps
    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))

def MAP(mat1, mat2):
  (k1, v1) = mat1
  (k2, v2) = mat2
  (i, _) = k1
  (_, k) = k2
  yield ((i, k), v1 * v2)

def REDUCE(key, values):
  yield (key, sum(values))

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]
print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))  # should return true

240 key-value pairs were sent over a network.
[[0.20998052 0.65628081 0.25757578 0.70159683 0.41067737 0.12276101
  0.80991423 0.56710402 0.64995187 0.73290727 0.50445792 0.37058196
  0.49116715 0.26601853 0.53148777 0.83365928 0.70914297 0.44699584
  0.59214382 0.66858384 0.64818951 0.62622691 0.44216521 0.92507785
  0.47585046 0.44532643 1.02169325 0.36828388 0.31508834 0.55441683
  0.84618442 0.56359079 0.80205485 0.71772654 0.79573597 0.75547552
  0.9921628  1.01039953 0.26671794 0.1295454 ]
 [0.32071925 0.83102822 0.48457356 0.93098719 0.69279204 0.28087653
  1.10051664 0.89762645 0.92686509 0.84915194 0.62580713 0.66995399
  0.57880329 0.4350904  0.76451447 1.12043481 0.84648878 0.60465183
  0.90809407 0.85005936 0.7589293  0.80545359 0.741465   1.23713278
  0.62679402 0.51182647 1.43838229 0.58994058 0.40499181 0.82887991
  1.1178903  0.7082604  0.9658665  1.04185289 1.01100932 0.90870467
  1.33950861 1.39085502 0.48159326 0.27213535]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [141]:
Да, решение будет работать.

SyntaxError: invalid syntax (<ipython-input-141-7c1926266fe9>, line 1)